# Setting Up

In [ ]:
# !pip install pyomo
# !pip install gurobipy
# !pip install xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import itertools
import pyomo.environ as pe
import pyomo.gdp as gdp
import gurobipy
#import pyomo.opt as opt

# Data

## Sets

In [ ]:
list(range(1, 11))

In [ ]:
sets = {
    # "S"     : ["x1", "x2", ...],  # Indices for supply geogpraphies
    # "X"     : ["x1", "x2", ...],  # Indices for plant site locations
    # "M"     : ["m1", "m2", ...],  # Indices for markets
    "T"       : [1],  # Indices for periods (10 years)
    "F"       : ["Bstraw", "Wstraw"],  # Indices for feedstocks
    "C"       : [],  # Indices for components
    "C_Solid" : ["Cellulose", "Hemicellulose", "Lignin"],  # Indices for solid components
    "C_Liquid": ["Xylose", "Glucose"],  # Indices for liquid components
    
    "BP"      : ['Succinic Acid'
                 #,'Glycolic Acid','Formic Acid','Acetic Acid','Phenolic','Furfural','HMF'
                ],  # Indices for by-products
    
    "U"       : ['H2SO4 kg'
                 #,'Water kg','Ammonia kg','Ionic Liquid mat. kg','Lime mat. kg'
                ],  # Indices for utilities
    
    "K"       : [],  # Indices for process options
    "K_prep"  : ['MILL3mm'
                 #,'MILL1.4mm','MILL10mm','MILL0.853mm','MILL0.15mm'
                ],  # Indices for preprocessing options
    
    "K_pret"  : ['Dilute Acid','LHW','AFEX','Steam Explosion','Lime','Ionic Liquid'],  # Indices for pretreatment options
    
    "K_conv"  : ['SHF','SSF','SHCF','SSCF'],  # Indices for fermentation options
    
    "K_pur"   : ["Reactive Extraction", 'Precipitation', 'Electrodialysis', 'Direct Crystalization', 'Azeotropic Dist', 'Extraction', 'Absorption'],  # Indices for purification options
    
    "J"       : ["Strain1","Strain2","Strain3","Strain4","Strain5","Strain6 ","Strain7","Strain8","Strain9" 
                 #"8G-3", "SPT2OE", "Y2034", "Angle", "CRD51", "MD4", "TP50", "SR8/TFP3-XI76", "SR8/TFP3-XI76"
                ],  # Indices for fermentation strains
    
    "P"       : ['Ethanol', 'Lactic Acid', 'Succinic Acid'],  # Indices for final products
    
    "Q"       : [10**6]  # Indices for technology capacity levels
}

sets["C"] = sets["C_Solid"]  + sets["C_Liquid"] 
sets["K"] = sets["K_prep"]  + sets["K_pret"] + sets["K_conv"] + sets["K_pur"]

## Parameters

Most parameters are indexed across different sets. To improve decision-maker interaction, the parametric tables over different sets are created with all possible combinations of the sets but empty values for the parameter and exported as excel files for easy inputs

In [ ]:
# Funciton for create the dataframe from the cartesian product of specific sets
def create_parametric_df(sets):
    df = pd.DataFrame(itertools.product(*sets.values()), columns=sets.keys())
    # drop duplicated rows
    df = df.drop_duplicates().reset_index(drop=True)
    # Create an empty value columns
    df.loc[:, 'Value'] = np.nan
    # Input 0 in impossible combinations (relevant for yields as you can't convert cellulose into cellulose)
    # Identify rows where a value in a column is equal to a value in another column for all columns
    equal_values_rows = df[df.apply(lambda row: len(set(row)) < len(row), axis = 1)]
    # Cahnge value for identified rows
    df.loc[equal_values_rows.index, "Value"] = 0
    return df

def create_compatability_dic():
    
    # Dictionary to store the DataFrames
    dfs = {}

    # Compatability Preprocessing with Pretreatment
    dfs['Compatibility_Pretreatment'] = create_parametric_df({'K_prep' : sets['K_prep'], 'K_pret' : sets['K_pret']})
    
    # Compatability Pretreatment, Conversion, Fermentation Strain, Component and Product   
    dfs['Conversion_Matrix_Pret_Conv'] = create_parametric_df({'K_pret': sets['K_pret'], 'K_conv': sets['K_conv']})
    dfs['Conversion_Matrix_C_Conv'] = create_parametric_df({'C': sets['C'], 'K_conv': sets['K_conv']})
    dfs['Conversion_Matrix_Conv_J'] = create_parametric_df({'K_conv': sets['K_conv'], 'J': sets['J']})
    dfs['Conversion_Matrix_J_P'] = create_parametric_df({'J': sets['J'], 'P': sets['P']})
    dfs['Conversion_Matrix_C_P'] = create_parametric_df({'C': sets['C'], 'P': sets['P']})
    dfs['Conversion_Matrix_Conv_P'] = create_parametric_df({'K_conv': sets['K_conv'], 'P': sets['P']})
    
    dfs['Compatibility_Purification'] = create_parametric_df({'P': sets['P'], 'K_pur': sets['K_pur']})

    return dfs

def create_parametric_dic(compat_dfs):
    # Dictionary to store the DataFrames
    dfs = {}

    # Generate DataFrames and store them in the dictionary
    dfs['Cost_of_Technology'] = create_parametric_df({'K': sets['K'], 'Q': sets['Q']})
    dfs['MinMax_Flows'] = create_parametric_df({'Flow': ['IN', 'OUT'], 'MinMax': ['Min', 'Max'], 'K': sets['K'], 'Q': sets['Q']})
    dfs['Utility_Consumption'] = create_parametric_df({'U': sets['U'], 'K': sets['K']})
    dfs['Byproduct_Production'] = create_parametric_df({'BP': sets['BP'], 'K': sets['K']})
    dfs['Feedstock_Composition'] = create_parametric_df({'C': sets['C'], 'F': sets['F']})
    
    dfs['Compatibility_Pretreatment'] = compat_dfs['Compatibility_Pretreatment']
    dfs['Yield_Pretreatment'] = create_parametric_df({'C': sets['C'], 'K_pret': sets['K_pret']})
    
    # Compatability Conversion
    df = pd.merge(compat_dfs['Conversion_Matrix_Pret_Conv'], compat_dfs['Conversion_Matrix_C_Conv'], on = "K_conv", how = "outer")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_Conv_J'], on = "K_conv", how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_J_P'], on = "J", how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_C_P'], on = ["C", "P"], how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)
    df = df.merge(compat_dfs['Conversion_Matrix_Conv_P'], on = ["K_conv", "P"], how = "left")
    df["Value"] = df.Value_x * df.Value_y
    df = df[df["Value"] != 0].drop(['Value_x', 'Value_y'], axis = 1)

    df = df[['C', 'K_pret', 'K_conv', 'J', 'P', 'Value']]

    dfs['Compatibility_Conversion'] = df
    
    dfs['Yield_Hydrolysis'] = df[['K_conv', 'K_pret', 'C', 'P', 'Value']].drop_duplicates().reset_index(drop=True).sort_values(['K_conv', 'K_pret', 'C', 'P'])
    dfs['Yield_Conversion'] = df[[ 'P', 'C', 'K_conv', 'J', 'Value']].drop_duplicates().reset_index(drop=True).sort_values(['J', 'P', 'C', 'K_conv'])
    
    dfs['Solid_Loading'] = create_parametric_df({'K_conv' : sets['K_conv']})

    dfs['Compatibility_Purification'] = compat_dfs['Compatibility_Purification']
    
    dfs['Yield_Purification'] = dfs['Compatibility_Purification'][dfs['Compatibility_Purification'].Value != 0].copy()
    dfs['Depreciation_Factor'] = create_parametric_df({'T' : sets['T']})
    dfs['Operating_Cost'] = dfs['Depreciation_Factor'].copy()
    dfs['Maintenance_Cost'] = dfs['Depreciation_Factor'].copy()
    dfs['Demand'] = create_parametric_df({'P': sets['P'], 'T': sets['T']})
    dfs['Supply'] = create_parametric_df({'F': sets['F'], 'T': sets['T']})
    dfs['Price_F'] = create_parametric_df({'F': sets['F'], 'T': sets['T']})
    dfs['Price_U'] = create_parametric_df({'U': sets['U'], 'T': sets['T']})
    dfs['Price_P'] = create_parametric_df({'P': sets['P'], 'T': sets['T']})

    return dfs

def create_compatability_excel(xls_path = "CompatabilityMatrices.xlsx"):
    dfs = create_compatability_dic()
    writer = pd.ExcelWriter(xls_path, engine='xlsxwriter')
    for name, df in dfs.items():
        df.to_excel(writer, sheet_name=name, index = False)
    writer.close()
    writer.handles = None
    return dfs

def read_compatability_excel(xls_path = "CompatabilityMatricesInputs.xlsx"):

    dfs = create_compatability_dic()
    excel_dfs = {}
    
    # Read excel
    xls = pd.ExcelFile(xls_path)
    for name, df in dfs.items():
        excel_df = pd.read_excel(xls, sheet_name=name)
        try:
            excel_df = excel_df.drop("Unnamed: 0", axis = 1)
        except:
            pass
        
        # concat excel df with full df
        df = pd.concat([excel_df, dfs[name]]).reset_index(drop=True)
        # keep first appearence (coming from excel df) of duplicated values in all columns except Value
        df = df.drop_duplicates(subset = df.columns.difference(['Value']), keep='first').reset_index(drop=True)
        # change all NA to 0 excpet for solid loading
        df.Value = 1 if name in ["Solid_Loading"] else df.Value.fillna(0)
        
        excel_dfs[name] = df
    
    xls.close()
    xls.handles = None
    
    return excel_dfs

def create_parametric_excel(xls_path = "ParametricTables.xlsx", compat = None):
    
    if compat is None:
        # Try to get input tables
        print("No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx")
        compat_dfs = read_compatability_excel()
    elif type(compat) is str:
        compat_dfs = read_compatability_excel(compat)
    else:
        compat_dfs = compat
    
    dfs = create_parametric_dic(compat_dfs)
    writer = pd.ExcelWriter(xls_path, engine='xlsxwriter')
    for name, df in dfs.items():
        df.to_excel(writer, sheet_name=name, index = False)
    writer.close()
    writer.handles = None
    return dfs

def read_parametric_excel(xls_path, compat = None):
    
    if compat is None:
        # Try to get input tables
        print("No compatabiity tables given. Attempting to retrieve from CompatabilityMatricesInputs.xlsx")
        compat_dfs = read_compatability_excel()
    elif type(compat) is str:
        compat_dfs = read_compatability_excel(compat)
    else:
        compat_dfs = compat
        
    # Create empty dic to force parameters to be 0 when user accidently forgets certain inputs
    dfs = create_parametric_dic(compat_dfs)
    
    # Read excel
    excel_dfs = {}
    xls = pd.ExcelFile(xls_path)
    for name, df in dfs.items():
        excel_df = pd.read_excel(xls, sheet_name=name)
        try:
            excel_df = excel_df.drop("Unnamed: 0", axis = 1)
        except:
            pass
        # concat excel df with full df
        df = pd.concat([excel_df, dfs[name]]).reset_index(drop=True)
        # keep first appearence (coming from excel df) of duplicated values in all columns except Value
        df = df.drop_duplicates(subset = df.columns.difference(['Value']), keep='first').reset_index(drop=True)
        # change all NA to 0 excpet for solid loading
        df.Value = 1 if name in ["Solid_Loading"] else df.Value.fillna(0)
        
        excel_dfs[name] = df
    xls.close()
    xls.handles = None
    return excel_dfs

In [ ]:
dfs = create_parametric_excel("ParametricTables-V1.xlsx")
print(dfs['Yield_Conversion'].shape)
dfs['Yield_Conversion'].head()

In [ ]:
dfs = read_parametric_excel("ParametricTablesInputs-V1.xlsx")
print(dfs['Yield_Conversion'].shape)
dfs['Yield_Conversion'][dfs['Yield_Conversion'].Value != 0].head()

In [ ]:
dfs['Yield_Hydrolysis'].head()

# Model

## Initialiazation

### Model

In [ ]:
model = pe.ConcreteModel()

### Sets

In [ ]:
# Initialize the sets based on the 'sets' dictionary
model.T = pe.Set(initialize=sets['T'])  # Indices for periods
model.F = pe.Set(initialize=sets['F'])  # Indices for feedstocks
model.C = pe.Set(initialize=sets['C'])  # Indices for components
model.C_Solid = pe.Set(initialize=sets['C_Solid'])  # Indices for solid components
model.C_Liquid = pe.Set(initialize=sets['C_Liquid'])  # Indices for liquid components
model.BP = pe.Set(initialize=sets['BP'])  # Indices for by-products
model.U = pe.Set(initialize=sets['U'])  # Indices for utilities
model.K = pe.Set(initialize=sets['K'])  # Indices for process options
model.K_prep = pe.Set(initialize=sets['K_prep'])  # Indices for preprocessing options
model.K_pret = pe.Set(initialize=sets['K_pret'])  # Indices for pretreatment options
model.K_conv = pe.Set(initialize=sets['K_conv'])  # Indices for fermentation options
model.K_pur = pe.Set(initialize=sets['K_pur'])  # Indices for purification options
model.J = pe.Set(initialize=sets['J'])  # Indices for fermentation strains
model.P = pe.Set(initialize=sets['P'])  # Indices for final products
model.Q = pe.Set(initialize=sets['Q'])  # Indices for technology capacity levels

# Auxiliary Sets
model.Flow = pe.Set(initialize=["IN", "OUT"])
model.MinMax = pe.Set(initialize=["Min", "Max"])
model.FCP = pe.Set(initialize=list(sets['F']) + list(sets['C']) + list(sets['P']))

### Parameters

In [ ]:
dfs = read_parametric_excel("ParametricTablesInputs-V1.xlsx")

# Cost_of_Technology
parameter_dict = dfs['Cost_of_Technology'].set_index(['K', 'Q']).Value.to_dict()
model.Cost_of_Technology = pe.Param(model.K, model.Q, initialize=parameter_dict, default=0)

# MinMax_Flows
parameter_dict = dfs['MinMax_Flows'].set_index(['Flow', 'MinMax', 'K', 'Q']).Value.to_dict()
model.MinMax_Flows = pe.Param(model.Flow, model.MinMax, model.K, model.Q, initialize=parameter_dict, default=0)

# Utility_Consumption
parameter_dict = dfs['Utility_Consumption'].set_index(['U', 'K']).Value.to_dict()
model.Utility_Consumption = pe.Param(model.U, model.K, initialize=parameter_dict, default=0)

# Byproduct_Production
parameter_dict = dfs['Byproduct_Production'].set_index(['BP', 'K']).Value.to_dict()
model.Byproduct_Production = pe.Param(model.BP, model.K, initialize=parameter_dict, default=0)

# Compatibility_Pretreatment
parameter_dict = dfs['Compatibility_Pretreatment'].set_index(['K_prep', 'K_pret']).Value.to_dict()
model.Compatibility_Pretreatment = pe.Param(model.K_prep, model.K_pret, initialize=parameter_dict, default=0)

# Feedstock_Composition
parameter_dict = dfs['Feedstock_Composition'].set_index(['C', 'F']).Value.to_dict()
model.Feedstock_Composition = pe.Param(model.C, model.F, initialize=parameter_dict, default=0)

# Yield_Pretreatment
parameter_dict = dfs['Yield_Pretreatment'].set_index(['C', 'K_pret']).Value.to_dict()
model.Yield_Pretreatment = pe.Param(model.C, model.K_pret, initialize=parameter_dict, default=0)

# Compatibility_Conversion
parameter_dict = dfs['Compatibility_Conversion'].set_index(['C', 'K_pret', 'K_conv', 'J', 'P']).Value.to_dict()
model.Compatibility_Conversion = pe.Param(model.C, model.K_pret, model.K_conv, model.J, model.P, initialize=parameter_dict, default=0)

# Solid_Loading
parameter_dict = dfs['Solid_Loading'].set_index(['K_conv']).Value.to_dict()
model.Solid_Loading = pe.Param(model.K_conv, initialize=parameter_dict, default=1)

# Yield_Hydrolysis
parameter_dict = dfs['Yield_Hydrolysis'].set_index(['K_conv', 'K_pret', 'C', 'P']).Value.to_dict()
model.Yield_Hydrolysis = pe.Param(model.K_conv, model.K_pret, model.C, model.P, initialize=parameter_dict, default=0)

# Yield_Conversion
parameter_dict = dfs['Yield_Conversion'][['P', 'C', 'K_conv', 'J', "Value"]].set_index(['P', 'C', 'K_conv', 'J']).Value.to_dict()
model.Yield_Conversion = pe.Param(model.P, model.C, model.K_conv, model.J, initialize=parameter_dict, default=0)

# Compatibility_Purification
parameter_dict = dfs['Compatibility_Purification'].set_index(['P', 'K_pur']).Value.to_dict()
model.Compatibility_Purification = pe.Param(model.P, model.K_pur, initialize=parameter_dict, default=0)

# Yield_Purification (same structure as Compatibility_Purification)
parameter_dict = dfs['Yield_Purification'].set_index(['P', 'K_pur']).Value.to_dict()
model.Yield_Purification = pe.Param(model.P, model.K_pur, initialize=parameter_dict, default=0)

# Depreciation_Factor
parameter_dict = dfs['Depreciation_Factor'].set_index(['T']).Value.to_dict()
model.Depreciation_Factor = pe.Param(model.T, initialize=parameter_dict, default=0)

# Operating_Cost (same structure as Depreciation_Factor)
parameter_dict = dfs['Operating_Cost'].set_index(['T']).Value.to_dict()
model.Operating_Cost = pe.Param(model.T, initialize=parameter_dict, default=0)

# Maintenance_Cost (same structure as Depreciation_Factor)
parameter_dict = dfs['Maintenance_Cost'].set_index(['T']).Value.to_dict()
model.Maintenance_Cost = pe.Param(model.T, initialize=parameter_dict, default=0)

# Demand
parameter_dict = dfs['Demand'].set_index(['P', 'T']).Value.to_dict()
model.Demand = pe.Param(model.P, model.T, initialize=parameter_dict, default=0)

# Supply
parameter_dict = dfs['Supply'].set_index(['F', 'T']).Value.to_dict()
model.Supply = pe.Param(model.F, model.T, initialize=parameter_dict, default=0)

# Price
parameter_dict = dfs['Price_F'].set_index(['F', 'T']).Value.to_dict()
model.Price_F = pe.Param(model.F, model.T, initialize=parameter_dict, default=0)
parameter_dict = dfs['Price_U'].set_index(['U', 'T']).Value.to_dict()
model.Price_U = pe.Param(model.U, model.T, initialize=parameter_dict, default=0)
parameter_dict = dfs['Price_P'].set_index(['P', 'T']).Value.to_dict()
model.Price_P = pe.Param(model.P, model.T, initialize=parameter_dict, default=0)

model.R = pe.Param(initialize = 0.02)
model.Tax = pe.Param(initialize = 0.3)

In [ ]:
# Initialize parameters as defined in dfs dictionary: doenst work yet, cant fin a way to initialize coorectly last line

# A function to determine the indexing sets of a given dataframe based on the columns
def determine_indexing_sets(df):
    return tuple(col for col in df.columns if col != "Value")

def initialize_parameters(parameter_dfs):
    # Loop through the paramneter dfs and initialize parameters
    for df_name, df in parameter_dfs.items():
        index_sets = determine_indexing_sets(df)
        # Pyomo only accepts dict {(colum A, B, C) : Value, next row}
        param_init_dict = df.set_index([*index_sets]).Value.to_dict()

        model_sets = tuple(getattr(model, index_set) for index_set in index_sets)
        setattr(model, df_name, pe.Param(model.P, model.T, initialize=param_init_dict, within='Any'))

#initialize_parameters(read_parametric_excel("ParametricTables.xlsx"))

## Defining Variables

### Binary Variables

In [ ]:
#########################
# Decision Variables
#########################

# Indicates if technology k with capacity q is chosen to be installed at period t
model.Y_tech = pe.Var(model.K, model.Q, model.T, domain=pe.Binary)

# Indicates if preprocessing option k_prep is chosen for feedstock f at period t
model.Y_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.Binary)
# Indicates if pretreatment option k_pret is chosen for feedstock f at period t
model.Y_pret = pe.Var(model.F, model.K_pret, model.T, domain=pe.Binary)
# Indicates if fermentation option k_conv with strain j is chosen to produce final product p with component c at period t
model.Y_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.Binary)
# Indicates if purification option k_pur is chosen for product p at period t
model.Y_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.Binary)

#########################
# Non-Decision Variables
#########################

# Indicates if tech k exists at period t
model.Y_K = pe.Var(model.K, model.T, domain=pe.Binary)

# Indicates if EBIT is positive at period t
#model.Y_EBIT = pe.Var(model.T, domain=pe.Binary)

### Continuous Variables

In [ ]:
# Decision Variables (inflows)

# Quantity of feedstock f going into preprocessing k_prep at period t
model.F_in_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.NonNegativeReals)

# Quantity of feedstock f going into pretreatment k_pret at period t
model.F_in_pret = pe.Var(model.F, model.K_pret, model.T, domain=pe.NonNegativeReals)

# Quantity of component c going into conversion k_conv with fermentation strain j at period t to produce product p
model.F_in_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.NonNegativeReals)

# Quantity of product p going into purification k_conv at period t
model.F_in_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.NonNegativeReals)

# Corresponding outflows

# Quantity of feedstock f going out of preprocessing k_prep at period t
model.F_out_prep = pe.Var(model.F, model.K_prep, model.T, domain=pe.NonNegativeReals)

# Quantity of component c going out of pretreatment k_pret at period t
model.F_out_pret = pe.Var(model.C, model.K_pret, model.T, domain=pe.NonNegativeReals)

# Quantity of product p going out of conversion k_conv with fermentation strain j at period t from component c
model.F_out_conv = pe.Var(model.C, model.P, model.K_conv, model.J, model.T, domain=pe.NonNegativeReals)

# Quantity of product p going out of purification k_pur at period t
model.F_out_pur = pe.Var(model.P, model.K_pur, model.T, domain=pe.NonNegativeReals)

# Other Relevant Variables

# Capital investment at period t
model.CAPEX = pe.Var(model.T, domain=pe.NonNegativeReals)

# Total flow of utility u consumed in process k at period t
model.F_util = pe.Var(model.U, model.K, model.T, domain=pe.NonNegativeReals)

# Total flow of byproduct bp generated in process k at period t
model.F_bypr = pe.Var(model.BP, model.K, model.T, domain=pe.NonNegativeReals)

# Liquid to add to biomass/ component c_sol to produce p with conversion k_conv in (SHF, SSF) at period t
model.Liq_bioreactor1 = pe.Var(model.C_Solid, model.P, model.K_conv, model.T, domain=pe.NonNegativeReals)

# Liquid to add or remove to biomass/ component c_sol + c_liq to produce p with conversion k_conv in (SHCF, SSCF) at period t
model.Liq_bioreactor2 = pe.Var(model.C_Solid, model.C_Liquid, model.P, model.K_conv, model.T, domain=pe.Reals) # Can be negative

# Aggregated Variables


# Total inbound flow entering process k at period t
model.F_in_process = pe.Var(model.K, model.T, domain=pe.NonNegativeReals)

# Total outbound flow exiting process k at period t
model.F_out_process = pe.Var(model.K, model.T, domain=pe.NonNegativeReals)

# Inbound flow of component c going in process k_conv at period t
model.F_in_conv_comp = pe.Var(model.C, model.K_conv, model.T, domain=pe.NonNegativeReals) 

# Outbound flow of component fcp out of process k_conv at period t
model.F_out_conv_prod = pe.Var(model.P, model.K_conv, model.T, domain=pe.NonNegativeReals)

# Total flow passing through technology k_pret at period t
#model.F_in_pret_total = pe.Var(model.K_pret, model.T, domain=pe.NonNegativeReals)

# Total flow passing through technology k_conv at period t
#model.F_in_conv_total = pe.Var(model.K_conv, model.T, domain=pe.NonNegativeReals)

# Total inbound flow of feedstock f at period t
model.F_in_feedstock = pe.Var(model.F, model.T, domain=pe.NonNegativeReals)

# Total consumed utility u at period t
model.F_in_utility = pe.Var(model.U, model.T, domain=pe.NonNegativeReals)

# Total produced byproduct bp at period t
model.F_out_byproduct = pe.Var(model.BP, model.T, domain=pe.NonNegativeReals)

# Total outbound flow of final product p at period t
model.F_out_product = pe.Var(model.P, model.T, domain=pe.NonNegativeReals)


In [ ]:
model.Yield_Conversion["Ethanol", "Cellulose", "SHF", "Strain1"]

## Constraint Formulation

### Technology and Capacity Allocation (1st Stage)

In [ ]:
# Aggregation of the flows for each process
model.flow_in_prep_total = pe.Constraint(model.K_prep, model.T, rule=lambda m, k, t: m.F_in_process[k, t] == sum(m.F_in_prep[f, k, t] for f in model.F))
model.flow_in_pret_total = pe.Constraint(model.K_pret, model.T, rule=lambda m, k, t: m.F_in_process[k, t] == sum(m.F_in_pret[f, k, t] for f in model.F))
model.flow_in_conv_total = pe.Constraint(model.K_conv, model.T, rule=lambda m, k, t: m.F_in_process[k, t] == sum(m.F_in_conv_comp[c, k, t] for c in model.C))
model.flow_in_pur_total = pe.Constraint(model.K_pur, model.T, rule=lambda m, k, t: m.F_in_process[k, t] == sum(m.F_in_pur[p, k, t] for p in model.P))

model.flow_out_prep_total = pe.Constraint(model.K_prep, model.T, rule=lambda m, k, t: m.F_out_process[k, t] == sum(m.F_out_prep[f, k, t] for f in model.F))
model.flow_out_pret_total = pe.Constraint(model.K_pret, model.T, rule=lambda m, k, t: m.F_out_process[k, t] == sum(m.F_out_pret[c, k, t] for c in model.C))
model.flow_out_conv_total = pe.Constraint(model.K_conv, model.T, rule=lambda m, k, t: m.F_out_process[k, t] == sum(m.F_out_conv_prod[p, k, t] for p in model.P))
model.flow_out_pur_total = pe.Constraint(model.K_pur, model.T, rule=lambda m, k, t: m.F_out_process[k, t] == sum(m.F_out_pur[p, k, t] for p in model.P))


In [ ]:
# Capacity Selection Disjunction

def rule_capacity_disjunct(disjunct, k, q, t, flag):
    m = disjunct.model()
    
    if flag:
        disjunct.c1 = pe.ConstraintList() 
        disjunct.c1.add(m.Y_tech[k, q, t] == 1)
        disjunct.c1.add(expr = m.MinMax_Flows["IN", "Min", k, q] <= m.F_in_process[k, t])
        disjunct.c1.add(expr = m.F_in_process[k, t] <= m.MinMax_Flows["IN", "Max", k, q])
        disjunct.c1.add(expr = m.MinMax_Flows["OUT", "Min", k, q] <= m.F_out_process[k, t])
        disjunct.c1.add(expr = m.F_out_process[k, t] <= m.MinMax_Flows["OUT", "Max", k, q])
    else:
        disjunct.c0 = pe.ConstraintList() 
        disjunct.c0.add(expr = m.Y_tech[k, q, t] == 0)
        disjunct.c0.add(expr = m.F_in_process[k, t] == 0)
        disjunct.c0.add(expr = m.F_out_process[k, t] == 0)

model.capacity_disjunct = gdp.Disjunct(model.K, model.Q, model.T, [0, 1], rule = rule_capacity_disjunct)

model.capacity_disjunction = gdp.Disjunction(model.K, model.Q, model.T, 
                                                rule = lambda m, k, q, t: [
                                                    model.capacity_disjunct[k, q, t, i] for i in [0, 1]
                                                ])

In [ ]:
# Capacity Disjunction (option 2). Requires Checking if implementation is correct

def create_capacity_disjuntion():
    model.capacity_disjunct1 = gdp.Disjunct(model.K, model.Q, model.T, 
                               rule=lambda m, k, q, t: m.Y_tech[k, q, t] == 1 and \
                                    m.MinMax_Flows["IN", "Min", k, q] <= m.F_in_process[k, t] <= m.MinMax_Flows["IN", "Max", k, q] and \
                                    m.MinMax_Flows["OUT", "Min", k, q] <= m.F_out_process[k, t] <= m.MinMax_Flows["OUT", "Max", k, q])

    model.capacity_disjunct2 = gdp.Disjunct(model.K, model.Q, model.T, rule=lambda m, k, q, t: m.Y_tech[k, q, t] == 0 and \
                                    m.F_in_process[k, t] == 0 and m.F_out_process[k, t] == 0)


    model.capacity_disjunction = gdp.Disjunction(model.K, model.Q, model.T, 
                                    rule = lambda m, k, q, t: [m.capacity_disjunct1[k, q, t], m.capacity_disjunct2[k, q, t]])

# create_capacity_disjuntion()

In [ ]:
# At least one technology must exist at each time t
model.tech_existance = pe.Constraint(model.T, 
                                  rule = lambda m, t: sum(m.Y_K[k, t] for k in model.K) >= 1)

# If a technology is selected at time t, it remains until the end of T
model.tech_continuity_prep = pe.Constraint(model.K_prep, model.Q, model.T, 
                                    rule = lambda m, k, q, t: m.Y_tech[k, q, t+1] >= m.Y_tech[k, q, t] if t < max(model.T) else pe.Constraint.Skip)
model.tech_continuity_pret = pe.Constraint(model.K_pret, model.Q, model.T, 
                                    rule = lambda m, k, q, t: m.Y_tech[k, q, t+1] >= m.Y_tech[k, q, t] if t < max(model.T) else pe.Constraint.Skip)
model.tech_continuity_conv = pe.Constraint(model.K_conv, model.Q, model.T, 
                                    rule = lambda m, k, q, t: m.Y_tech[k, q, t+1] >= m.Y_tech[k, q, t] if t < max(model.T) else pe.Constraint.Skip)
model.tech_continuity_pur = pe.Constraint(model.K_pur, model.Q, model.T, 
                                    rule = lambda m, k, q, t: m.Y_tech[k, q, t+1] >= m.Y_tech[k, q, t] if t < max(model.T) else pe.Constraint.Skip)



# Only one capacity per technology at each time
model.capacity_selection = pe.Constraint(model.K, model.T, 
                                      rule = lambda m, k, t: sum(m.Y_tech[k, q, t] for q in model.Q) <= 1)
model.capacity_selection_total = pe.Constraint(model.K, model.T, 
                                      rule = lambda m, k, t: m.Y_K[k, t] == sum(m.Y_tech[k, q, t] for q in model.Q))


# CAPEX calculation
model.capex_calculation = pe.Constraint(model.T,
                                      rule = lambda m, t: m.CAPEX[t] == sum(sum((m.Cost_of_Technology[k, q] * (m.Y_tech[k, q, t] - m.Y_tech[k, q, t - 1]) if t > 1 else 0) for k in model.K) for q in model.Q))
model.capex_calculation_initial = pe.Constraint(
    rule = lambda m: m.CAPEX[1] == sum(m.Cost_of_Technology[k, q] * m.Y_tech[k, q, 1] for k in model.K for q in model.Q))


### Operational Constraints (2nd Stage): General

In [ ]:
# Choosing a determined process for feedstock, components or products, only if it exists at time t

# Choose preprocessing for feedstock f only if technology k_prep exists at time t
model.prep_constraint = pe.Constraint(model.F, model.K_prep, model.T, 
                                  rule=lambda m, f, k_prep, t: m.Y_prep[f, k_prep, t] <= m.Y_K[k_prep, t])

# Choose pretreatment for feedstock f only if technology k_pret exists at time t
model.pret_constraint = pe.Constraint(model.F, model.K_pret, model.T, 
                                  rule=lambda m, f, k_pret, t: m.Y_pret[f, k_pret, t] <= m.Y_K[k_pret, t])

# Choose conversion for component c to produce product p only if technology k_conv exists at time t
model.conv_constraint = pe.Constraint(model.C, model.P, model.K_conv, model.J, model.T, 
                                  rule=lambda m, c, p, k_conv, j, t: m.Y_conv[c, p, k_conv, j, t] <= m.Y_K[k_conv, t])

# Choose purification for product p only if technology k_pur exists at time t
model.pur_constraint = pe.Constraint(model.P, model.K_pur, model.T, 
                                 rule=lambda m, p, k_pur, t: m.Y_pur[p, k_pur, t] <= m.Y_K[k_pur, t])


# If a technology is selected at time t, it remains until the end of T
model.tech_total_prep = pe.Constraint(model.K_prep, model.T, 
                                  rule=lambda m, k_prep, t: sum(m.Y_prep[f, k_prep, t] for f in m.F) == m.Y_K[k_prep, t])
model.tech_total_pret = pe.Constraint(model.K_pret, model.T, 
                                  rule=lambda m, k_pret, t: sum(m.Y_pret[f, k_pret, t] for f in m.F) == m.Y_K[k_pret, t])
model.tech_total_conv = pe.Constraint(model.K_conv, model.T, 
                                  rule=lambda m, k_conv, t: sum(sum(sum(m.Y_conv[c, p, k_conv, j, t] for j in m.J) for p in m.P) for c in m.C) == m.Y_K[k_conv, t])
model.tech_total_pur = pe.Constraint(model.K_pur, model.T, 
                                 rule=lambda m, k_pur, t: sum(m.Y_pur[p, k_pur, t] for p in m.P) == m.Y_K[k_pur, t])


In [ ]:
# Calculation of utility consumption and byproducts generation

# Utility consumption per processed flow in technology k at t
model.utility_consumption_prep = pe.Constraint(model.U, model.K_pret, model.T, 
                                      rule=lambda m, u, k_pret, t: m.F_util[u, k_pret, t] == sum(m.F_in_pret[f, k_pret, t] * m.Utility_Consumption[u, k_pret] for f in model.F))
model.utility_consumption_conv = pe.Constraint(model.U, model.K_conv, model.T, 
                                      rule=lambda m, u, k_conv, t: m.F_util[u, k_conv, t] == sum(m.F_in_conv_comp[c, k_conv, t] * m.Utility_Consumption[u, k_conv] for c in model.C))

# Byproduct generation per processed flow in technology k at t
#model.byproduct_generation = pe.Constraint(model.BP, model.K, model.T, 
#                                       rule=lambda m, bp, k, t: m.F_bypr[bp, k, t] == sum(m.F_in_fcp[fcp, k, t] * m.Byproduct_Production[bp, k] for fcp in model.FCP) if k in model.K_prep or k in model.K_conv else pe.Constraint.Skip)


In [ ]:
model.Supply.pprint()

### Operational Constraints (2nd Stage): Preprocessing

In [ ]:
# Preprocessing Technology Disjunction

def rule_preprocessing_disjunct(disjunct, f, k_prep, t, flag):
    m = disjunct.model()
    
    if flag:
        disjunct.c1 = pe.ConstraintList() 
        disjunct.c1.add(m.Y_prep[f, k_prep, t] == 1)
        disjunct.c1.add(expr = m.F_in_prep[f, k_prep, t] <= m.Supply[f, t])
    else:
        disjunct.c0 = pe.ConstraintList() 
        disjunct.c0.add(expr = m.Y_prep[f, k_prep, t] == 0)
        disjunct.c0.add(expr = m.F_in_prep[f, k_prep, t] == 0)

model.preprocessing_disjunct = gdp.Disjunct(model.F, model.K_prep, model.T, [0, 1], rule = rule_preprocessing_disjunct)

model.preprocessing_disjunction = gdp.Disjunction(model.F, model.K_prep, model.T, 
                                                rule = lambda m, f, k_prep, t: [
                                                    model.preprocessing_disjunct[f, k_prep, t, i] for i in [0, 1]
                                                ])

In [ ]:
# Choose only one process per feedstock:
model.single_prep_constraint = pe.Constraint(model.F, model.T, 
                                             rule = lambda m, f, t: sum(m.Y_prep[f, k_prep, t] for k_prep in model.K_prep) <= 1)

# Mass Balance
model.prep_mass_balance = pe.Constraint(model.F, model.K_prep, model.T, 
                                        rule=lambda m, f, k_prep, t: m.F_out_prep[f, k_prep, t] == m.F_in_prep[f, k_prep, t])


### Operational Constraints (2nd Stage): Pretreatment

In [ ]:
# Pretreatment Technology Disjunction

def rule_pretreatment_disjunct(disjunct, f, k_pret, t, flag):
    m = disjunct.model()
    
    if flag:
        disjunct.c1 = pe.ConstraintList() 
        disjunct.c1.add(m.Y_pret[f, k_pret, t] == 1)
        disjunct.c1.add(expr = m.F_in_pret[f, k_pret, t] <= sum(m.Compatibility_Pretreatment[k_prep, k_pret] * m.F_out_prep[f, k_prep, t] for k_prep in m.K_prep))
    else:
        disjunct.c0 = pe.ConstraintList() 
        disjunct.c0.add(expr = m.Y_pret[f, k_pret, t] == 0)
        disjunct.c0.add(expr = m.F_in_pret[f, k_pret, t] == 0)

model.pretreatment_disjunct = gdp.Disjunct(model.F, model.K_pret, model.T, [0, 1], rule = rule_pretreatment_disjunct)

model.pretreatment_disjunction = gdp.Disjunction(model.F, model.K_pret, model.T, 
                                                rule = lambda m, f, k_pret, t: [
                                                    model.pretreatment_disjunct[f, k_pret, t, i] for i in [0, 1]
                                                ])

In [ ]:
# Choose only one process per feedstock:
model.single_pret_constraint = pe.Constraint(model.F, model.T, 
                                             rule = lambda m, f, t: sum(m.Y_pret[f, k_pret, t] for k_pret in model.K_pret) <= 1)

# Transformation
model.pret_transformation = pe.Constraint(model.F, model.K_pret, model.C, model.T,
                                          rule = lambda m, f, k_pret, c, t: 
                                          m.F_out_pret[c, k_pret, t] == m.F_in_pret[f, k_pret, t] * m.Feedstock_Composition[c, f] * m.Yield_Pretreatment[c, k_pret])

# Mass Balance
#model.pret_mass_balance = pe.Constraint(model.F, model.K_pret, model.T, 
#                                        rule=lambda m, f, k_pret, t: 
#                                        sum(m.F_in_pret[f, k_pret, t] for f in model.F) + sum(m.F_util[u, k_pret, t] for u in model.U) == sum(m.F_out_pret[c, k_pret, t] for c in model.C) + sum(m.F_bypr[bp, k_pret, t] for bp in model.BP))


### Operational Constraints (2nd Stage): Conversion

In [ ]:
# Conversion Technology Disjunction

def rule_conversion_disjunct(disjunct, c, p, k_conv, j, t, flag):
    m = disjunct.model()
    
    if flag:
        disjunct.c1 = pe.ConstraintList() 
        disjunct.c1.add(m.Y_conv[c, p, k_conv, j, t] == 1)
        disjunct.c1.add(expr = m.F_in_conv[c, p, k_conv, j, t] <= sum(m.Compatibility_Conversion[c, k_pret, k_conv, j, p] * m.F_out_pret[c, k_pret, t]  * m.Yield_Hydrolysis[k_conv, k_pret, c, p] for k_pret in m.K_pret))
    else:
        disjunct.c0 = pe.ConstraintList() 
        disjunct.c0.add(expr = m.Y_conv[c, p, k_conv, j, t] == 0)
        disjunct.c0.add(expr = m.F_in_conv[c, p, k_conv, j, t] == 0)

model.conversion_disjunct = gdp.Disjunct(model.C, model.P, model.K_conv, model.J, model.T, [0, 1], rule = rule_conversion_disjunct)

model.conversion_disjunction = gdp.Disjunction(model.C, model.P, model.K_conv, model.J, model.T,
                                                rule = lambda m, c, p, k_conv, j, t: [
                                                    m.conversion_disjunct[c, p, k_conv, j, t, i] for i in [0, 1]
                                                ])

In [ ]:
# Solid Loading

# Liquid calculation for solid components and specific conversion processes
#model.sl_bioreactor_1 = pe.Constraint(
 #   model.C_Solid, model.P, model.K_conv, model.T,
  #  rule=lambda m, c_sol, p, k_conv, t: 
   # m.Liq_bioreactor1[c_sol, p, k_conv, t] == (1 - m.Solid_Loading[k_conv]) / m.Solid_Loading[k_conv] * sum(m.F_in_conv[c_sol, p, k_conv, j, t] for j in m.J if k_conv in ['SHF', 'SSF']))

# Liquid calculation for solid and liquid components, for specific conversion processes
#model.sl_bioreactor_2 = pe.Constraint(
 #   model.C_Solid, model.C_Liquid, model.P, model.K_conv, model.T,
  #  rule=lambda m, c_sol, c_liq, p, k_conv, t: 
   # m.Liq_bioreactor2[c_sol, c_liq, p, k_conv, t] == (1 - m.Solid_Loading[k_conv]) / m.Solid_Loading[k_conv] * sum(m.F_in_conv[c_sol, p, k_conv, j, t] for j in m.J if k_conv in ['SHCF', 'SSCF']) - sum(m.F_in_conv[c_liq, p, k_conv, j, t] for j in m.J if k_conv in ['SHCF', 'SSCF']))

In [ ]:
# Choose only one fermentation strain per feedstock:
model.single_conv_constraint = pe.Constraint(model.C, model.P, model.K_conv, model.J, model.T, 
                                             rule = lambda m, c, p, k_conv, j, t: sum(m.Y_conv[c, p, k_conv, j, t] for j in model.J) <= 1)

# Transformation
model.conv_transformation = pe.Constraint(model.C, model.P, model.K_conv, model.J, model.T,
                                          rule = lambda m, c, p, k_conv, j, t: 
                                          m.F_out_conv[c, p, k_conv, j, t] == m.F_in_conv[c, p, k_conv, j, t] * m.Yield_Conversion[p, c, k_conv, j])


# Mass Balances

model.conv_total_component = pe.Constraint(model.C, model.P, model.K_conv, model.J, model.T,
                                        rule = lambda m, c, p, k_conv, j, t: 
                                        m.F_in_conv_comp[c, k_conv, t] == sum(sum(m.F_in_conv[c, p, k_conv, j, t] for j in model.J) for p in model.P))
model.conv_total_product = pe.Constraint(model.C, model.P, model.K_conv, model.J, model.T,
                                        rule = lambda m, c, p, k_conv, j, t: 
                                        m.F_out_conv_prod[p, k_conv, t] == sum(sum(m.F_out_conv[c, p, k_conv, j, t] for j in model.J) for c in model.C))

#model.conv_mass_balance = pe.Constraint(model.K_conv, model.T, 
#                                        rule=lambda m, k_conv, t: 
#                                        sum(m.F_in_conv_comp[c, k_conv, t] for c in model.C) + sum(m.F_util[u, k_conv, t] for u in model.U) == sum(m.F_out_conv_prod[p, k_conv, t] for p in model.P) + sum(m.F_bypr[bp, k_conv, t] for bp in model.BP))


### Operational Constraints (2nd Stage): Purification

In [ ]:
# Purification Technology Disjunction

def rule_purification_disjunct(disjunct, p, k_pur, t, flag):
    m = disjunct.model()
    
    if flag:
        disjunct.c1 = pe.ConstraintList() 
        disjunct.c1.add(m.Y_pur[p, k_pur, t] == 1)
        disjunct.c1.add(expr = m.F_in_pur[p, k_pur, t] <= m.Compatibility_Purification[p, k_pur] * sum(m.F_out_conv_prod[p, k_conv, t] for k_conv in m.K_conv))
    else:
        disjunct.c0 = pe.ConstraintList() 
        disjunct.c0.add(expr = m.Y_pur[p, k_pur, t] == 0)
        disjunct.c0.add(expr = m.F_in_pur[p, k_pur, t] == 0)

model.purification_disjunct = gdp.Disjunct(model.P, model.K_pur, model.T,[0, 1], rule = rule_purification_disjunct)

model.purification_disjunction = gdp.Disjunction(model.P, model.K_pur, model.T,
                                                rule = lambda m, p, k_pur, t: [
                                                    m.purification_disjunct[p, k_pur, t, i] for i in [0, 1]
                                                ])

In [ ]:
# Choose only one process per product:
model.single_pur_constraint = pe.Constraint(model.P, model.T, 
                                             rule = lambda m, p, t: sum(m.Y_pur[p, k_pur, t] for k_pur in model.K_pur) <= 1)

# Transformation
model.pur_transformation = pe.Constraint(model.P, model.K_pur, model.C, model.T,
                                          rule = lambda m, p, k_pur, c, t: 
                                          m.F_out_pur[p, k_pur, t] == m.F_in_pur[p, k_pur, t] * m.Yield_Purification[p, k_pur])

# Mass Balance
#model.pur_mass_balance = pe.Constraint(model.P, model.K_pur, model.T, 
#                                        rule=lambda m, p, k_pur, t: 
#                                        m.F_in_pur[p, k_pur, t] == m.F_out_pur[p, k_pur, t])


### Total Biorefinery Statistics

In [ ]:

# Total feedstock input is equal to the sum of all feedstocks entering each pretreatment technology
model.total_feedstock = pe.Constraint(model.F, model.T, 
                                          rule=lambda m, f, t: m.F_in_feedstock[f, t] == sum(m.F_in_prep[f, k_prep, t] for k_prep in model.K_prep))

# Total utility input is equal to the sum of all utilities consumed in all processes
model.total_utility = pe.Constraint(model.U, model.T, 
                                         rule=lambda m, u, t: m.F_in_utility[u, t] == sum(m.F_util[u, k, t] for k in model.K))

# Total byproduct output is the sum of all byproducts generated in all processes
model.total_byproduct = pe.Constraint(model.BP, model.T, 
                                           rule=lambda m, bp, t: m.F_out_byproduct[bp, t] == sum(m.F_bypr[bp, k, t] for k in model.K))

# Total product output is equal to the sum of all products exiting each purification technology
model.total_product = pe.Constraint(model.P, model.T, 
                                         rule=lambda m, p, t: m.F_out_product[p, t] == sum(m.F_out_pur[p, k_pur, t] for k_pur in model.K_pur))

# Feedstock input should not exceed its supply at each time period
model.feedstock_supply_constraint = pe.Constraint(model.F, model.T, 
                                              rule=lambda m, f, t: m.F_in_feedstock[f, t] <= m.Supply[f, t])

# Product output should not exceed its demand at each time period
model.product_demand_constraint = pe.Constraint(model.P, model.T, 
                                            rule=lambda m, p, t: m.F_out_product[p, t] <= m.Demand[p, t])


## Objective Functions

In [ ]:
model.EBIT = pe.Var(model.T, domain = pe.Reals)
model.EBIT_calc = pe.Constraint(model.T, 
                                rule=lambda m, t: 
                                m.EBIT[t] == sum(m.F_out_product[p, t] * m.Price_P[p, t] for p in model.P) \
                                - sum(m.F_in_feedstock[f, t] * m.Price_F[f, t] for f in model.F) \
                                - sum(m.F_in_utility[u, t] * m.Price_U[u, t] for u in model.U) \
                                - (m.Operating_Cost[t] + m.Maintenance_Cost[t])* m.CAPEX[t])

In [ ]:
def NPV_rule(m):      
    return sum( ((1-m.Tax) * m.EBIT[t] - m.CAPEX[t]) /(1 + m.R)**t for t in m.T)
model.Objective_function = pe.Objective(rule = NPV_rule, sense = pe.maximize) 

# Model Solve

In [ ]:
#Solving the model and displaying the solution
M_value = 10**10
pe.TransformationFactory('gdp.bigm').apply_to(model, bigM=M_value)

In [ ]:
solver_parameters = "ResultFile=iismodel.ilp" # write an ILP file to print the IIS

In [ ]:
opt = pe.SolverFactory("gurobi", solver_io="python") 
opt.options['NonConvex'] = 2
opt.options['DualReductions'] = 0
results = opt.solve(model, options_string=solver_parameters)
results.write()

In [ ]:
var_mapping = {}
index = 1  # Start index at 1, assuming Gurobi's first variable is x1

for v in model.component_objects(pe.Var, active=True):
    for index_within_v in v:
        var_mapping[f"x{index}"] = f"{v.name}[{index_within_v}]"
        index += 1

with open('var_mapping.json', 'w') as fp:
    json.dump(var_mapping, fp)
    
constraint_mapping = {}
index = 1  # Start index at 1, assuming Gurobi's first variable is x1

for v in model.component_objects(pe.Constraint, active=True):
    for index_within_v in v:
        constraint_mapping[f"x{index}"] = f"{v.name}[{index_within_v}]"
        index += 1

In [ ]:
# Sort the dictionary by length of the keys in descending order
var_mapping = dict(sorted(var_mapping.items(), key=lambda item: len(item[0]), reverse=True))
constraint_mapping = dict(sorted(constraint_mapping.items(), key=lambda item: len(item[0]), reverse=True))

# Read the file content
with open("iismodel.ilp", "r") as file:
    iis_output = file.read()

# Replace Gurobi names with Pyomo names in constraints
for gurobi_name, pyomo_name in constraint_mapping.items():
    iis_output = iis_output.replace(f"{gurobi_name}:", f"{pyomo_name}:")

# Replace Gurobi names with Pyomo names
for gurobi_name, pyomo_name in var_mapping.items():
    iis_output = iis_output.replace(gurobi_name, pyomo_name)

# Write the modified content back to the file
with open("iismodel-translated.ilp", "w") as file:
    file.write(iis_output)

print("File updated successfully!")
#print(iis_output)

In [ ]:
# Read the file content
with open("model.ilp", "r") as file:
    iis_output = file.read()

# Replace Gurobi names with Pyomo names
for gurobi_name, pyomo_name in var_mapping.items():
    iis_output = iis_output.replace(gurobi_name, pyomo_name)

# Write the modified content back to the file
with open("model.ilp", "w") as file:
    file.write(iis_output)

print("File updated successfully!")

In [ ]:
model.Y_tech.pprint()

In [ ]:
constraint_mapping['x3224']

In [ ]:
def print_vars():
    for v in model.component_objects(pe.Var, active=True):
        try:
            varobject = getattr(model, str(v))
            print("Variable", v)
            for index in varobject:
                if varobject[index].value is not None:
                    if varobject[index].value > 0:
                        print("   ", index, varobject[index].value) 
        except:
            pass
#print_vars()

In [ ]:
pe.value(model.Objective_function)

In [ ]:
model.tech_total_prep

In [ ]:
model.Compatibility_Purification.pprint()

In [ ]:
model.F_out_conv.pprint()

In [ ]:
F_out_conv[('Hemicellulose', 'Lactic Acid', 'SSCF', 'Strain1', 1)]